# Independencies (You can run all below from here)

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# The code below is also needed for using Google Colab
# BEFORE executing this, you must place the mingpt folder supplied in the assignment
# your google drive, within the folder "Colab Notebooks"
#
# It mounts and changes into the folder that contains mingpt, which you must upload to google drive
# So un-comment it if you've uploaded mingpt to your google drive, into the  "Colab Notebooks" folder

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from google.colab import drive
import matplotlib.pyplot as plt
%pip install datasets
%pip install transformers
%pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch 
import numpy as np
import pandas as pd
import pyarrow as pa
from torch.optim import AdamW
from transformers import get_scheduler
from tqdm.auto import tqdm
import evaluate
from datasets import Dataset
from nltk.tokenize import sent_tokenize 
from pathlib import Path 
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from transformers import AutoModelForSequenceClassification

# Using the trained transformer model to make prediction

In [ ]:
%cd /content
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
hf_model = AutoModelForSequenceClassification.from_pretrained("sshleifer/tiny-gpt2", num_labels = 2)
checkpoint = torch.load('transformer.pt', DEVICE)
hf_model.load_state_dict(checkpoint)
hf_model.to(DEVICE)

/content


Some weights of the model checkpoint at sshleifer/tiny-gpt2 were not used when initializing GPT2ForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at sshleifer/tiny-gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 2)
    (wpe): Embedding(1024, 2)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((2,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((2,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((2,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout):

In [ ]:
# Show the uploaded fils in /content/
!ls

1019_UkraineCombinedTweetsDeduped.csv  sample_data     tw.csv
Prediction_of_tweets.csv	       transformer.pt


## Elon musk Tweet prediction

### Choose the input file you want the model to make predictions on

1.   Upload the tweet file to colab
2.   set 'input_file_name' to the correct file name


In [ ]:
tweet_df = pd.read_csv("./tw.csv") 
tweet_df.drop(columns=['id', 'created_at', 'retweet_count', 'reply_count', 'like_count'], inplace = True)
#tweet_df['label'] = np.nan
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    16 non-null     object
dtypes: object(1)
memory usage: 256.0+ bytes


In [ ]:
from datasets import Dataset
dataset_tweet = Dataset(pa.Table.from_pandas(tweet_df))
tokenizer = AutoTokenizer.from_pretrained("sshleifer/tiny-gpt2")
tokenizer.pad_token = tokenizer.eos_token
hf_model.config.pad_token_id = hf_model.config.eos_token_id
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True)
tokenized_datasets_tweet = dataset_tweet.map(tokenize_function, batched=True)
tokenized_datasets_tweet.set_format("torch")
tokenized_datasets_tweet = tokenized_datasets_tweet.remove_columns(["text", "attention_mask"])
#tokenized_datasets_tweet = tokenized_datasets_tweet.rename_column("label", "labels")

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tweet_dataloader = DataLoader(tokenized_datasets_tweet, batch_size=1)
tweet_df['prediction'] = np.nan
hf_model.eval()
i = 0
for batch in tweet_dataloader:
        batch = {k: v.to(DEVICE) for k, v in batch.items()}
        with torch.no_grad():
            outputs = hf_model(**batch)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        tweet_df.at[i, 'prediction'] = predictions.item()
        i += 1
tweet_df.head(10)

,text,prediction
0,Let’s try this then: the will of the people wh...,0.0
1,"Also worth noting that a possible, albeit unli...",0.0
2,This is highly likely to be the outcome in the...,0.0
3,Ukraine-Russia Peace:\n\n- Redo elections of a...,0.0
4,@bdquinn @LindseyGrahamSC Kherson and Zaporizh...,1.0
5,"@LindseyGrahamSC Obviously, not everyone who v...",0.0
6,@LindseyGrahamSC Below is the electoral map of...,0.0
7,@LindseyGrahamSC Assuming you believe that the...,0.0
8,@KonstantinKisin I do not think it is simple a...,0.0
9,@ZelenskyyUa I still very much support Ukraine...,0.0


save the prediction dataframe as csv file to colab

In [ ]:
tweet_df.to_csv("Prediction_of_tweets" + ".csv", index=False)

## Ukraine War Tweet prediction

### Choose the input file you want the model to make predictions on

1.   Upload the tweet file to colab
2.   set 'input_file_name' to the correct file name



In [ ]:
input_file_name = '1019_UkraineCombinedTweetsDeduped.csv'

In [ ]:
ukraine_df = pd.read_csv("./" + input_file_name) 
ukraine_df = pd.DataFrame(ukraine_df['text'], index = ukraine_df.index)

dataset_ukraine = Dataset(pa.Table.from_pandas(ukraine_df))

tokenized_datasets_ukraine = dataset_ukraine.map(tokenize_function, batched=True)
tokenized_datasets_ukraine.set_format("torch")
tokenized_datasets_ukraine = tokenized_datasets_ukraine.remove_columns(["text", "attention_mask"])
ukraine_df.info()

  0%|          | 0/63 [00:00<?, ?ba/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62979 entries, 0 to 62978
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    62979 non-null  object
dtypes: object(1)
memory usage: 492.1+ KB


In [ ]:
ukraine_dataloader = DataLoader(tokenized_datasets_ukraine, batch_size=1)
ukraine_df['prediction'] = np.nan
hf_model.eval()
i =0
for batch in ukraine_dataloader:
        batch = {k: v.to(DEVICE) for k, v in batch.items()}
        with torch.no_grad():
            outputs = hf_model(**batch)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        ukraine_df.at[i, 'prediction'] = predictions.item()
        i += 1
ukraine_df.head(20)

,text,prediction
0,"Comisión Europea desembolsa 2,000 millones de ...",0.0
1,"Animal shelter Dogs and Cats, we need your hel...",0.0
2,"Ukraine war map as of Oct. 18 with #Kherson, D...",0.0
3,"Welcome to our shelter!\nLocated in Ukraine, K...",1.0
4,Does the #RussiaUkraine conflict bode trouble ...,0.0
5,🇺🇦La guerra de #Ucrania ha conmocionado al mun...,0.0
6,#NATO\n#北大西洋条約機構\nは東西冷戦中の1949年に🇺🇸など西側諸国が、旧ソ連など...,0.0
7,"Now Ukraine Peace March, Berlin 27. March 202...",1.0
8,"🚨📢| Con drones suicidas, Rusia ataca Ucrania; ...",0.0
9,Institutional investors bet big on Solana whil...,0.0


save the prediction dataframe as csv file to colab

In [ ]:
ukraine_df.to_csv(input_file_name[:4] + "_Prediction_of_Ukrain_war" + ".csv", index=False)